# 前期环境搭建
    1.复制虚拟机node1,
        A)删除网卡,
        B)删除多余的hadoop,环境变量,保存jdk
        C)配置本地源
    
    2.网络环境
        vi /etc/sysconfig/network  //配置主机名
        vi /etc/hosts              //配置好node1,node2,node3,node4
    3.防火墙关闭,selinux 
        service iptables stop
        chkconfig iptables off
        setenforce 0    //getenforce 查看
        vi /etc/selinux/config (SELINUX=disabled)
    4.安装JDK
    5.安装NTP
         yum -y install ntp
         chkconfig ntp on
         ntpdate ntp1.aliyun.com  //修改时间服务器地址
    6.安装配置mysql
         yum -y install mysql-server
         mysql -uroot -p
         chkconfig mysqld on
             
        GRANT ALL ON *.* TO 'root'@'%' IDENTIFIED BY '123' WITH GRANT OPTION;
        flush privileges;
        delete from user where user='root' and host='localhost'; //删除root其他权限
    7.下载第三方依赖包
     yum -y install chkconfig python bind-utils psmisc libxslt zlib sqlite cyrus-sasl-plain cyrus-sasl-gssapi fuse、fuse-libs redhat-lsb
     
    8.最后克隆出node2,node3,node4来,改IP,hostname,
         vi /etc/sysconfig/network
     
    9.ssh免密钥,实现任何节点的互通!:
        所有节点生成密钥,添加信任到node1
        node1分发authorized到node2,node3,node4
        
        ssh-keygen  //生成密钥
        ssh-copy-id -i .ssh/id_rsa.pub node3  //让node3添加信任

# CDH安装(帐号:mckj_zhangxk@23a0673F@)

    1.CDH(5.4.0)软件包下载地址(http://archive.cloudera.com/cdh5/parcels/5.4.0/)
    2.CDH manager(5.4.3)下载地址:http://archive.cloudera.com/cm5/cm/5/cloudera-manager-el6-cm5.4.3_x86_64.tar.gz
    
    步骤
    1)在node1上面,解压cloudera-manager
    
      mkdir /root/cloudera-manager
      tar -xvf cloudera-manager.tar -C /root/cloudera-manager
    2)对所有节点,创建用户cloudera-scm,不给shell,不给home
    useradd --system --no-create-home --shell=/bin/false  cloudera-scm
    3)配置agent
        修改:cm-5.4.3/etc/cloudera-scm-agent/config.ini中server_host=node1
    
    4)配置manager-server数据库
       手动 拷贝mysql.jar文件到目录 /usr/share/java/,注意jar包名称要修改为mysql-connector-java.jar
       grant all on *.* to 'cloudera_scm'@'%' identified by '123' with grant option;
       
       在cm-5.4.3/share/cmf/schema/目录下面
       scm_prepare_database.sh mysql cloudera_scm -h node01 -ucloudera_scm -p123 --scm-host node01 scm scm scm
       格式如下:
       scm_prepare_database.sh <数据库类型> <数据库名> <数据库服务器地址> <用户名> <密码> <cm_server服务器地址>
     
    5)创建Parcel目录
      Server节点:
          A).mkdir /root/cloudera/parcel-repo
          B).chown cloudera-scm:cloudera-scm /root/cloudera/parcel-repo
          C).拷贝CDH软件安装包到/root/cloudera/parcel-repo,(3个文件)
      
      Agent节点
          A).mkdir -p /root/cloudera/parcels
          B).chown cloudera-scm:cloudera-scm /root/cloudera/parcels
    6)配置环境变量:
            PATH=PATH:cloudera-manager/cm-5.4.3/etc/init.d/
            启动server
                cloudera-scm-server start
            启动server agent   
                cloudera-scm-agent start
    7)http://node1:7180